In [1]:
!pip install transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 30.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 1.2 MB 32.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=3800ae73eba33fe15e2a7ce984a7b5a43f521d242206d3dc38ce1305e75b58a8
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [2]:
import pandas as pd
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
from transformers import DistilBertTokenizerFast

In [3]:
# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:

# initialize the tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.to(device)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [37]:
df = pd.read_csv('context_ques_ans.csv')

FileNotFoundError: ignored

In [8]:
context = list(df['Context'])
ques = list(df['Question'])
ans = list(df['Answer'])

In [9]:
#creating dictionary for Finetuning the bert
ans_dict = []
idx = 0 
for a in ans : 
    temp_dict = {}
    temp_dict['text'] = a
    start_idx = context[idx].lower().find(a.lower())
    end_idx = start_idx + len(a)
    temp_dict['answer_start'] = start_idx
    temp_dict['answer_end'] = end_idx
    if start_idx > end_idx or start_idx <0 or end_idx <0 :
      print(start_idx, end_idx, context[idx])
    ans_dict.append(temp_dict)
    if context[idx].lower().find(a.lower()) == -1 :
        print(ans[idx])
    idx+=1

# tokenize
train_encodings = tokenizer(context, ques, truncation=True, padding=True)


In [10]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        #print(answers)
        #break
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, ans_dict)

In [11]:

class createDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = createDataset(train_encodings)

#### Fine Tuning

In [12]:
import re
import os
import spacy
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np


In [15]:
with open('Mahabharta_dataset.txt', 'r') as file:
    file_content_raw = file.read()

In [16]:
# splitting passage-wise
passages = file_content_raw.split('\n')

In [17]:
# returns the cleaned text
def cleaned_text(text):
    text = text.strip()
    text = re.sub(r'\n', ' ', text)
#     text = re.sub("[^\w]", ' ', text)
    text = text.replace(r'[^0-9a-zA-Z:,]+','')
    text = text.strip()
    return text

In [18]:
passages = [cleaned_text(passage) for passage in passages]
passages = list(filter(lambda x: len(x) > 20, passages))

In [19]:
passages_all = []
def create_passages(passages) : 
  for para in passages :
    lines = para.split('.')
    passages_all.append(para)
    for l in lines : 
      passages_all.append(l)
  
create_passages(passages)

In [20]:
embedding_dict = {}

In [21]:
# creating passage dictionary
passage_dict_all = {
    i : passage for i, passage in zip(range(len(passages_all)), passages_all)
}

In [24]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

#########################################
#                                       #
# SET 'model_flag' to TRUE for Model 1  #
#       and False for model 2.          #
#                                       #
#########################################
model_flag = True

if model_flag == True:
  model_similarity = SentenceTransformer('nli-distilroberta-base-v2') #APPROACH 2
else :
  model_similarity = SentenceTransformer('bert-base-nli-mean-tokens') #APPROACH 3


Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
#calculating embeddings for each passage

for id in passage_dict_all  :
  embedding_dict[id] = model_similarity.encode(passage_dict_all[id])
  if id %1000 == 0 :
    print(id)

In [30]:
embedding_list = []

for key in embedding_dict : 
  embedding_list.append(embedding_dict[key])

In [31]:
def retrieve_relevant_doc_list(embedding,thresh = 0.2, n = 100 ):
  temp_ans = [ ]
  similarity = cosine_similarity([embedding],embedding_list)[0]
  #print(similarity)
  for j in range(len(similarity)) :
    if similarity[j] > thresh :
      temp_ans.append([similarity[j] , j ])
  ans = []
  temp_ans.sort(reverse = True)
  for i in range(min(len(temp_ans) , n)) : 
    ans.append(temp_ans[i][1])
  
  return ans


In [ ]:
%%time
for i in range(125) :
  
  ques_embedding = model_similarity.encode(ques[i])
  
  docs = retrieve_relevant_doc_list(ques_embedding, n = 10)

  print('QUES : ' , ques[i], '\nTRUE ANS :' , ans[i])

  
  count = 0 
  

  for d in docs:
    passage = passage_dict_all[d]
    input_ids = tokenizer.encode(cleaned_text(ques[i]), passage)

    sep_idx = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_idx+1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    if len(input_ids) > 500 :
      continue 
    output = model(torch.tensor([input_ids]).to(device),  token_type_ids=torch.tensor([segment_ids]).to(device))
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end <= num_seg_a and answer_start <= num_seg_a:
      continue
    if answer_end > answer_start:
        answer = " ".join(tokens[answer_start:answer_end+1])
        answer = tokens[answer_start]
        for k in range(answer_start+1, answer_end+1):
            if tokens[k][0:2] == "##":
                answer += tokens[k][2:]
            else:
                answer += " " + tokens[k]
        print(count,'th ANSWER :' , answer)
        count+=1
    

In [34]:
def answerMyQues(ques) : 
  ques_embedding = model_similarity.encode(ques)
  
  docs = retrieve_relevant_doc_list(ques_embedding, n = 10)

  
  
  count = 0 
  

  for d in docs:
    passage = passage_dict_all[d]
    input_ids = tokenizer.encode(cleaned_text(ques), passage)

    sep_idx = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_idx+1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    if len(input_ids) > 500 :
      continue 
    output = model(torch.tensor([input_ids]).to(device),  token_type_ids=torch.tensor([segment_ids]).to(device))
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end <= num_seg_a and answer_start <= num_seg_a:
      continue
    if answer_end > answer_start:
        answer = " ".join(tokens[answer_start:answer_end+1])
        answer = tokens[answer_start]
        for k in range(answer_start+1, answer_end+1):
            if tokens[k][0:2] == "##":
                answer += tokens[k][2:]
            else:
                answer += " " + tokens[k]
        return answer
        count+=1

In [ ]:
# for USERs

In [36]:


while(True):
  print('Answer: ',answerMyQues(input("Enter the question: ")))
  print('\n')
  if input('Do you want to ask more question? (y/n): ') != 'y':
    break

Enter the question: who was the teacher of bhishma
Answer:  bhishma


Do you want to ask more question? (y/n): y
Enter the question: who was the island born
Answer:  dwaipayana


Do you want to ask more question? (y/n): y
Enter the question: who three princess of kashi
Answer:  amba , ambika , and ambalika


Do you want to ask more question? (y/n): y
Enter the question: who was the father of pandavas
Answer:  child who would become the future father of the pandavas


Do you want to ask more question? (y/n): y
Enter the question: who was pandu
Answer:  one with a white complexion


Do you want to ask more question? (y/n): y
Enter the question: who was dhritrashtra
Answer:  great hero


Do you want to ask more question? (y/n): y
Enter the question: why gandhari blindfold herself
Answer:  when gandhari was married to dhritarastra


Do you want to ask more question? (y/n): n
